## ORL_RE_Inference

Use the trained model to predict input sentences and bring them into a format accepted by the relation classifier.

In [1]:
import os
# DataSource in SSA-format
DATASOURCE="../../etl/data/processed/Perin_Preprocessing/01_test.json"
DATADEST="../../etl/data/processed/ORL_RE_Inference/01_test.json"

# DATASOURCE="../../etl/data/processed/Perin_Preprocessing/01_train.json"
# DATADEST="../../etl/data/processed/ORL_RE_Inference/01_train.json"

# DATASOURCE="../../etl/data/processed/REConverter/01_train_re.json"

VERIFY_RE_MODEL=False

# CONFIGS for NER
# CONFIG-VARIABLES
ORL_MODEL_PATH="../ORL/data/trained_model_german_bert"

# CONFIGS for RE
# GS_INPUT_DIR = "/content/drive/MyDrive/odsc-2022-data/NYT"
DATA_DIR = "../../etl/data/processed/REConverter"
splits = ["train", "test", "val"]
BASE_MODEL_NAME = "bert-base-german-cased"
# BASE_MODEL_NAME = "bert-base-cased"
MODEL_DIR = os.path.join("../RelationClassifier/data", "{:s}-re".format(BASE_MODEL_NAME))

### Read-In the data

Reading in the data so it can be processed. The format is SSA.

In [2]:
# makedirs if not exist
os.makedirs("../../etl/data/processed/ORL_RE_Inference", exist_ok=True)

In [3]:
opinion_interface = {'Source': [[], []], 
             'Target': [[], []], 
             'Polar_expression': [[], []], 
             'Polarity': None, 
             'Intensity': 'Average'}

In [4]:
import json
import copy

out_data = None
gold_polarities = []
def convert_ssa_to_sents(datasource):
    with open(datasource, "r", encoding="utf-8") as f:
        data = json.load(f)
    out_data = copy.deepcopy(data)
    sents = []
    for i, rec in enumerate(data):
        out_data[i]["opinions"] = [copy.deepcopy(opinion_interface)]
        # print(rec)
        sents.append(rec["text"])
        gold_polarities.append(rec["opinions"][0]["Polarity"])
    return sents, out_data

ssa_sents, out_data = convert_ssa_to_sents(DATASOURCE)

In [5]:
print(out_data)

[{'sent_id': '0', 'text': 'Andere Proteste entstanden wegen der seit Jahren miserablen Wirtschaftslage und der allumfassenden Korruption im Land und blieben meist regional .', 'opinions': [{'Source': [[], []], 'Target': [[], []], 'Polar_expression': [[], []], 'Polarity': None, 'Intensity': 'Average'}]}, {'sent_id': '1', 'text': 'Flavia Schlittler Im Frühling 2015 wurde Nepal von verheerenden Erdbeben heimgesucht .', 'opinions': [{'Source': [[], []], 'Target': [[], []], 'Polar_expression': [[], []], 'Polarity': None, 'Intensity': 'Average'}]}, {'sent_id': '2', 'text': 'Der Konflikt sorgte innerhalb der CVP , in der man in normalen Zeiten ungern öffentlich streitet , für erheblichen Ärger .', 'opinions': [{'Source': [[], []], 'Target': [[], []], 'Polar_expression': [[], []], 'Polarity': None, 'Intensity': 'Average'}]}, {'sent_id': '3', 'text': 'Damals zerstörte ein Feuer das historische Gipfelhotel aus dem Jahr 1893 – ein grosser Verlust .', 'opinions': [{'Source': [[], []], 'Target': [[

### Target Holder PolarExpression Extraction

**Extract Target-Holder-Expression-Pairs**

In [6]:
# Load the ORL model.
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

model_path = ORL_MODEL_PATH

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)

pipe = pipeline(task="token-classification",
                # model=trainer.model, -- in case freshly trained
                model=model,
                tokenizer=tokenizer,
                aggregation_strategy="first"
               )

classified = pipe("Der Mann erschiesst seine Frau.")

print(classified)

[{'entity_group': 'LABEL_0', 'score': 0.99897814, 'word': 'Der', 'start': 0, 'end': 3}, {'entity_group': 'LABEL_1', 'score': 0.99242157, 'word': 'Mann', 'start': 4, 'end': 8}, {'entity_group': 'LABEL_3', 'score': 0.99666035, 'word': 'erschiesst', 'start': 9, 'end': 19}, {'entity_group': 'LABEL_0', 'score': 0.9993655, 'word': 'seine', 'start': 20, 'end': 25}, {'entity_group': 'LABEL_2', 'score': 0.9913374, 'word': 'Frau', 'start': 26, 'end': 30}, {'entity_group': 'LABEL_0', 'score': 0.999114, 'word': '.', 'start': 30, 'end': 31}]


In [7]:
from typing import NamedTuple

class PAS(NamedTuple):
    arg1: str
    arg2: str
    pexp: str

In [8]:
label_mapper = {
    "LABEL_0": "NEUTRAL",
    "LABEL_1": "HOLDER",
    "LABEL_2": "TARGET",
    "LABEL_3": "PEXP",
}

def extract_args(bert_output):
    """Obtain arguments from a dict-list of a BERT model."""
    # the holder is the first argument, the target is the second argument
    arg1=None
    arg2=None
    pexp=None
    for c in bert_output:
        if label_mapper[c["entity_group"]] == "HOLDER":
            arg1=c["word"]
        elif label_mapper[c["entity_group"]] == "TARGET":
            arg2=c["word"]
        elif label_mapper[c["entity_group"]] == "PEXP":
            pexp=c["word"]
        else:
            pass
        
    # if one is none, fill it by using the highest probability overall.

    return arg1, arg2, pexp

In [9]:
# Function doing the heavy lifting, here we assume one relation per sentence (no compositionality).
def extract_pas(sentence_string):
    # sentence with holder / target labels
    labelled_sentence = pipe(sentence_string)
    # obtain arguments from labels
    arg1, arg2, pexp = extract_args(labelled_sentence)
    return PAS(arg1, arg2, pexp)

print(extract_pas("Die Berliner Morgenpost hat den Angriff auf die Ukraine kritisiert."))

PAS(arg1='Morgenpost', arg2='Angriff', pexp='kritisiert')


### Generate unbreakable tokens based on input sentence + ORL inference

**Important**: We make sure that every sentence contains a dot in the end that is separated from the last word in the sentence!

In [10]:
# simplified_training_data
import re
import copy
from tqdm import tqdm

def identify_from_base_sent(target_word, base_sent):
    res = re.search(target_word, base_sent)
    return res.span()

def generate_head_based_entity_sentences(sentences, out_data=[]):
    entified_list = []
    ret_od = []
    for i, sent in tqdm(enumerate(sentences)):
        pas = extract_pas(sent)
        od = copy.deepcopy(out_data[i])
        base_sent = copy.deepcopy(sent)
        # print(pas)
        try:
            if pas.arg1:
                res = re.search(pas.arg1, sent)
                e1_s, e1_e = res.span()
                # print(out_data[i]["opinions"][0]["Source"])
                sent = sent[0:max(0, e1_s - 1)] + " <eSOURCE> " + sent[e1_s:min(len(sent), e1_e)] + " </eSOURCE> " + sent[min(len(sent), e1_e + 1):]
                
                e1_s, e1_e = identify_from_base_sent(pas.arg1, base_sent)
                od["opinions"][0].update({ "Source": [[base_sent[e1_s:min(len(sent), e1_e)]], [f"{str(e1_s)}:{str(e1_e)}"]] })

            if pas.arg2:
                res = re.search(pas.arg2, sent)
                e2_s, e2_e = res.span()
                sent = sent[0:max(0, e2_s - 1)] + " <eTARGET> " + sent[e2_s:min(len(sent), e2_e)] + " </eTARGET> " + sent[min(len(sent), e2_e + 1):]
                
                e2_s, e2_e = identify_from_base_sent(pas.arg2, base_sent)
                od["opinions"][0]["Target"] = [[base_sent[e2_s:min(len(sent), e2_e)]], [f"{str(e2_s)}:{str(e2_e)}"]]

            if pas.pexp:
                res = re.search(pas.pexp, sent)
                e3_s, e3_e = res.span()
                sent = sent[0:max(0, e3_s - 1)] + " <ePEXP> " + sent[e3_s:min(len(sent), e3_e)] + " </ePEXP> " + sent[min(len(sent), e3_e + 1):]
                
                e3_s, e3_e = identify_from_base_sent(pas.pexp, base_sent)
                od["opinions"][0]["Polar_expression"] = [[base_sent[e3_s:min(len(sent), e3_e)]], [f"{str(e3_s)}:{str(e3_e)}"]]


            for tag, expression in [("SOURCE", pas.arg1), ("TARGET", pas.arg2), ("PEXP", pas.pexp)]:
                if expression is None:
                    # res = re.search("(\.|\!)\s*?(<\/.*>)?\s*?$", sent)
                    res = re.search("(\.|\!)", sent)
                    if res is None:
                        sent += " ."
                    res = re.search("(\.|\!)", sent)
                    e_nf_s, e_nf_e = res.span()
                    #print(f"{ e_nf_s} ; {e_nf_e}")
                    sent = sent[0:max(0, e_nf_s - 1)] + f" <e{tag}> " + sent[e_nf_s:min(len(sent), e_nf_e)] + f" </e{tag}> " + sent[min(len(sent), e_nf_e + 1):]
                    
            # print(od[i]["opinions"][0])
            
            assert all([tag in sent for tag in ["<eTARGET>", "</eTARGET>", "<eSOURCE>", "</eSOURCE>", "<ePEXP>", "</ePEXP>"]]), "Not all tags present in sentence, falling back to trivial."

        except Exception as e:
            print(f"Something went wrong with extraction, {str(e)}, \n with sent {str(sent)} and sample # {i}")
            print("Trivialised!")
            # spaceholder
            # entified_list.append("Neutral <eSOURCE> . </eSOURCE> <eTARGET> . </eTARGET> <ePEXP> . </ePEXP>")
            od["text"] = "Neutral . .. ..."
            entified_list.append("Neutral <eSOURCE> . </eSOURCE> <eTARGET> .. </eTARGET> <ePEXP> ... </ePEXP>")
            od["opinions"][0].update({ "Source": [["."], ["8:9"]] })
            od["opinions"][0].update({ "Target": [[".."], ["10:12"]] })
            od["opinions"][0].update({ "Polar_expression": [["..."], ["13:16"]] })
            ret_od.append(od)
            continue

        entified_list.append(sent)
        # print(od)
        ret_od.append(od)
    return entified_list, ret_od

test_sents = ["Macron hat die Frau geliebt!",
"Jakob liebt Brigitte .",
 "Der Mann weiss nicht mehr wo er ist .",
 "Viele Personen sind von Haifischen ermordet worden .",
 "Liebe .",
 "Peter greift Michaela auf .",]

# entified_list = generate_head_based_entity_sentences()

# print(entified_list)
# don't forget stripping!
# print("\n")
# X = {"tokens": [e.strip().split() for e in entified_list]}
# print(X)

In [11]:
tagged_ssa_sents, out_data_w_entities = generate_head_based_entity_sentences(ssa_sents, out_data)

872it [00:27, 29.73it/s]

Something went wrong with extraction, Not all tags present in sentence, falling back to trivial., 
 with sent  < <eTARGET> S </eTARGET> URCE> Ähnliches </eSOURCE> <ePEXP> gilt </ePEXP> für die S 24 , die von Zürich über den Flughafen stündlich bis nach Thayngen und nach Weinfelden verkehrt . and sample # 866
Trivialised!


3775it [02:03, 32.30it/s]

Something went wrong with extraction, 'NoneType' object has no attribute 'span', 
 with sent Nach 25 Minuten sche <eTARGET> ter </eTARGET>  der Dortmunder <eSOURCE> Captain </eSOURCE> erstmals an ter Stegen , aus fünf Metern nur . and sample # 3769
Trivialised!


5323it [02:55, 30.40it/s]


In [12]:
tokenized_ssa_sents = {"tokens": [e.strip().split() for e in tagged_ssa_sents]}

In [13]:
out_data_w_entities

[{'sent_id': '0',
  'text': 'Andere Proteste entstanden wegen der seit Jahren miserablen Wirtschaftslage und der allumfassenden Korruption im Land und blieben meist regional .',
  'opinions': [{'Source': [['Proteste'], ['7:15']],
    'Target': [[], []],
    'Polar_expression': [['entstanden'], ['16:26']],
    'Polarity': None,
    'Intensity': 'Average'}]},
 {'sent_id': '1',
  'text': 'Flavia Schlittler Im Frühling 2015 wurde Nepal von verheerenden Erdbeben heimgesucht .',
  'opinions': [{'Source': [['Nepal'], ['41:46']],
    'Target': [['Erdbeben'], ['64:72']],
    'Polar_expression': [[], []],
    'Polarity': None,
    'Intensity': 'Average'}]},
 {'sent_id': '2',
  'text': 'Der Konflikt sorgte innerhalb der CVP , in der man in normalen Zeiten ungern öffentlich streitet , für erheblichen Ärger .',
  'opinions': [{'Source': [['Konflikt'], ['4:12']],
    'Target': [['Ärger'], ['115:120']],
    'Polar_expression': [['sorgte'], ['13:19']],
    'Polarity': None,
    'Intensity': 'Average'}

In [14]:
tokenized_ssa_sents

{'tokens': [['Andere',
   '<eSOURCE>',
   'Proteste',
   '</eSOURCE>',
   '<ePEXP>',
   'entstanden',
   '</ePEXP>',
   'wegen',
   'der',
   'seit',
   'Jahren',
   'miserablen',
   'Wirtschaftslage',
   'und',
   'der',
   'allumfassenden',
   'Korruption',
   'im',
   'Land',
   'und',
   'blieben',
   'meist',
   'regional',
   '<eTARGET>',
   '.',
   '</eTARGET>'],
  ['Flavia',
   'Schlittler',
   'Im',
   'Frühling',
   '2015',
   'wurde',
   '<eSOURCE>',
   'Nepal',
   '</eSOURCE>',
   'von',
   'verheerenden',
   '<eTARGET>',
   'Erdbeben',
   '</eTARGET>',
   'heimgesucht',
   '<ePEXP>',
   '.',
   '</ePEXP>'],
  ['Der',
   '<eSOURCE>',
   'Konflikt',
   '</eSOURCE>',
   '<ePEXP>',
   'sorgte',
   '</ePEXP>',
   'innerhalb',
   'der',
   'CVP',
   ',',
   'in',
   'der',
   'man',
   'in',
   'normalen',
   'Zeiten',
   'ungern',
   'öffentlich',
   'streitet',
   ',',
   'für',
   'erheblichen',
   '<eTARGET>',
   'Ärger',
   '</eTARGET>',
   '.'],
  ['Damals',
   '<ePEXP>',


### Delete this later!

Just for a test on the RE Model performance!

In [15]:
if VERIFY_RE_MODEL:
    with open(DATASOURCE, "r", encoding="utf-8") as f:
        tokenized_ssa_sents = {"tokens": [], "labels": []}
        for line in f:
            #print(line)
            ds = json.loads(line)
            tokenized_ssa_sents["tokens"].append(ds["tokens"])
            tokenized_ssa_sents["labels"].append(ds["label"])

    gold_polarities = [lbl for lbl in tokenized_ssa_sents["labels"]]

### Generate unbreakable tokens based on original dataframe

**Important**: We make sure that every sentence contains a dot in the end that is separated from the last word in the sentence!

In [16]:
import collections
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import random
import re
import shutil
import torch
import torch.nn as nn

from collections import Counter, defaultdict
from datasets import load_dataset, ClassLabel
from sklearn.metrics import (
    ConfusionMatrixDisplay, confusion_matrix, 
    classification_report, accuracy_score
)
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.optim import AdamW

from transformers import (
    BertTokenizerFast, BertModel, BertForPreTraining, BertConfig, BertPreTrainedModel,
    DataCollatorWithPadding,
    get_scheduler
)
from transformers.modeling_outputs import SequenceClassifierOutput

In [17]:
class BertForRelationExtraction(BertPreTrainedModel):
  def __init__(self, config, num_labels):
    super(BertForRelationExtraction, self).__init__(config)
    self.num_labels = num_labels
    # body
    self.bert = BertModel(config)
    # head
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
    #self.layer_norm = nn.LayerNorm(config.hidden_size * 2)
    #self.linear = nn.Linear(config.hidden_size * 2, self.num_labels)
    self.init_weights()
    
    self.layer_norm = nn.LayerNorm(config.hidden_size * 3)
    self.linear = nn.Linear(config.hidden_size * 3, self.num_labels)

  def forward(self, input_ids, token_type_ids, attention_mask,
              span_idxs, labels=None):
    outputs = (
        self.bert(input_ids, token_type_ids=token_type_ids,
                  attention_mask=attention_mask,
                  output_hidden_states=False)
            .last_hidden_state)
            
    sub_maxpool, obj_maxpool, pexp_maxpool = [], [], []
    #sub_maxpool, obj_maxpool = [], []
    for bid in range(outputs.size(0)):
      # span includes entity markers, maxpool across span
      sub_span = torch.max(outputs[bid, span_idxs[bid, 0]:span_idxs[bid, 1]+1, :], 
                           dim=0, keepdim=True).values
      obj_span = torch.max(outputs[bid, span_idxs[bid, 2]:span_idxs[bid, 3]+1, :],
                           dim=0, keepdim=True).values
      pexp_span = torch.max(outputs[bid, span_idxs[bid, 4]:span_idxs[bid, 5]+1, :], dim=0, keepdim=True).values
      sub_maxpool.append(sub_span)
      obj_maxpool.append(obj_span)
      pexp_maxpool.append(pexp_span)

    sub_emb = torch.cat(sub_maxpool, dim=0)
    obj_emb = torch.cat(obj_maxpool, dim=0)
    pexp_emb = torch.cat(pexp_maxpool, dim=0)
    rel_input = torch.cat((sub_emb, obj_emb, pexp_emb), dim=-1)
    #rel_input = torch.cat((sub_emb, obj_emb), dim=-1)

    rel_input = self.layer_norm(rel_input)
    rel_input = self.dropout(rel_input)
    logits = self.linear(rel_input)

    if labels is not None:
      loss_fn = nn.CrossEntropyLoss()
      loss = loss_fn(logits.view(-1, self.num_labels), labels.view(-1))
      return SequenceClassifierOutput(loss, logits)
    else:
      return SequenceClassifierOutput(None, logits)


# config = BertConfig.from_pretrained(BASE_MODEL_NAME)
# model = BertForRelationExtraction.from_pretrained(BASE_MODEL_NAME, 
#                                                   config=config,
#                                                   num_labels=len(valid_relations))
# model.bert.resize_token_embeddings(len(tokenizer.vocab))
# for batch in train_dl:
#   outputs = model(**batch)
#   break
# print("loss:", outputs.loss, "logits.size:", outputs.logits.size())

In [18]:
tokenizer = BertTokenizerFast.from_pretrained(BASE_MODEL_NAME)

vocab_size_orig = len(tokenizer.vocab)

marker_tokens = []
entity_types = ["eSOURCE", "eTARGET", "ePEXP"]
for entity_type in entity_types:
  marker_tokens.append("<{:s}>".format(entity_type))
  marker_tokens.append("</{:s}>".format(entity_type))

tokenizer.add_tokens(marker_tokens)
vocab_size_new = len(tokenizer.vocab)

print("original vocab size:", vocab_size_orig)
print("new vocab size:", vocab_size_new)

def encode_data(examples):
  tokenized_inputs = tokenizer(examples["tokens"],
                               is_split_into_words=True,
                               truncation=True)
  span_idxs = []
  for input_id in tokenized_inputs.input_ids:
    tokens = tokenizer.convert_ids_to_tokens(input_id)
    try:
        span_idxs.append([
          [idx for idx, token in enumerate(tokens) if token.startswith("<eS")][0],
          [idx for idx, token in enumerate(tokens) if token.startswith("</eS")][0],
          [idx for idx, token in enumerate(tokens) if token.startswith("<eT")][0],
          [idx for idx, token in enumerate(tokens) if token.startswith("</eT")][0],
          [idx for idx, token in enumerate(tokens) if token.startswith("<eP")][0],
          [idx for idx, token in enumerate(tokens) if token.startswith("</eP")][0],
        ])
    except Exception as e:
        print(f"Encountered error because of: { str(e) }, { str(tokens) }")
        # from IPython.core.debugger import Pdb; Pdb().set_trace()
        # print(f"Tokens of sentence: { str(examples['tokens']) }")

  tokenized_inputs["span_idxs"] = span_idxs
  # tokenized_inputs["labels"] = [label2id[label] for label in examples["label"]]
  return tokenized_inputs

#print(X)
#encoded = encode_data(X)
#encoded.keys()
#print(encoded)

original vocab size: 30000
new vocab size: 30006


In [19]:
encoded = encode_data(tokenized_ssa_sents)
# encoded

In [20]:
from datasets import Dataset
BATCH_SIZE = 4
collate_fn = DataCollatorWithPadding(tokenizer, padding="longest", return_tensors="pt")
test_dl = DataLoader(Dataset.from_dict(encoded),
                      # shuffle=True, 
                      batch_size=BATCH_SIZE, 
                      collate_fn=collate_fn)

# print([x for x in test_dl])

In [21]:
valid_relations = ["Positive", "Neutral", "Negative"]
valid_relations = sorted(list(valid_relations))

In [22]:
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint_file = os.path.join(MODEL_DIR, "ckpt-{:d}".format(3))

print("=> Loading checkpoint")
# checkpoint = model.from_pretrained(checkpoint_file)
config = BertConfig.from_pretrained(checkpoint_file)
model = BertForRelationExtraction.from_pretrained(checkpoint_file, 
                      config=config,
                      num_labels=len(valid_relations))
# model.load_state_dict(checkpoint["state_dict"])
model.bert.resize_token_embeddings(len(tokenizer.vocab))
model.eval()
model = model.to(device)

=> Loading checkpoint


In [23]:
# confirm the model is not in training
model.training

False

In [24]:
ytrue, ypred = [], []
with tqdm(test_dl, unit="batch") as tepoch:
    for batch in tepoch:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
          outputs = model(**batch)
          predictions = torch.argmax(outputs.logits, dim=-1).cpu().numpy()
          #labels = batch["labels"].cpu().numpy()
          #ytrue.extend(labels)
          ypred.extend(predictions)

#print("test accuracy: {:.3f}".format(accuracy_score(ytrue, ypred)))

#print(classification_report(ytrue, ypred, target_names=valid_relations))

100%|██████████████████████████████████████████████████████████| 1331/1331 [06:01<00:00,  3.68batch/s]


In [25]:
preds_full_label_text = [valid_relations[i] for i in ypred]

In [26]:
preds_full_label_text

['Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Positive',
 'Positive',
 'Neutral',
 'Neutral',
 'Positive',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Positive',
 'Neutral',
 'Neutral',
 'Positive',
 'Positive',
 'Neutral',
 'Negative',
 'Negative',
 'Neutral',
 'Neutral',
 'Positive',
 'Neutral',
 'Neutral',
 'Neutral',
 'Positive',
 'Neutral',
 'Neutral',
 'Negative',
 'Neutral',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Neutral',
 'Positive',
 'Neutral',
 'Negative',
 'Neutral',
 'Neutral',
 'Negative',
 'Neutral',
 'Neutral',
 'Negative',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Positive',
 'Positive',
 'Neutral',
 'Neutral',
 'Negative',
 'Neutral',
 'Neutral',
 'Negative',
 'Positive',
 'Neutral',
 'Positive',
 'Positive',
 'Neutral',
 'Neutral',
 'Positive',
 'Negative',
 'Neutral',
 'Negative',
 'Negative',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neg

In [27]:
valid_relations

['Negative', 'Neutral', 'Positive']

### Save the data

Reading in the data so it can be processed. The format is SSA.

In [28]:
def construct_ssa_dataset(preds, out_data_w_ents):
    new_out_struct = []
    for pred, out_ds in zip(preds, out_data_w_ents):
        od = copy.deepcopy(out_ds)
        od["opinions"][0]["Polarity"] = pred
        new_out_struct.append(od)
    return new_out_struct

out_data_w_entities_and_rel = construct_ssa_dataset(preds_full_label_text, out_data_w_entities)

In [29]:
out_data_w_entities_and_rel

[{'sent_id': '0',
  'text': 'Andere Proteste entstanden wegen der seit Jahren miserablen Wirtschaftslage und der allumfassenden Korruption im Land und blieben meist regional .',
  'opinions': [{'Source': [['Proteste'], ['7:15']],
    'Target': [[], []],
    'Polar_expression': [['entstanden'], ['16:26']],
    'Polarity': 'Neutral',
    'Intensity': 'Average'}]},
 {'sent_id': '1',
  'text': 'Flavia Schlittler Im Frühling 2015 wurde Nepal von verheerenden Erdbeben heimgesucht .',
  'opinions': [{'Source': [['Nepal'], ['41:46']],
    'Target': [['Erdbeben'], ['64:72']],
    'Polar_expression': [[], []],
    'Polarity': 'Neutral',
    'Intensity': 'Average'}]},
 {'sent_id': '2',
  'text': 'Der Konflikt sorgte innerhalb der CVP , in der man in normalen Zeiten ungern öffentlich streitet , für erheblichen Ärger .',
  'opinions': [{'Source': [['Konflikt'], ['4:12']],
    'Target': [['Ärger'], ['115:120']],
    'Polar_expression': [['sorgte'], ['13:19']],
    'Polarity': 'Neutral',
    'Intensi

In [30]:
# evaluate the polar expression performance
print("test accuracy: {:.3f}".format(accuracy_score(gold_polarities, preds_full_label_text)))

print(classification_report(gold_polarities, preds_full_label_text))

test accuracy: 0.572
              precision    recall  f1-score   support

    Negative       0.78      0.44      0.56      1774
     Neutral       0.48      0.86      0.62      1792
    Positive       0.67      0.41      0.51      1757

    accuracy                           0.57      5323
   macro avg       0.64      0.57      0.56      5323
weighted avg       0.64      0.57      0.56      5323



In [31]:
with open(DATADEST, "w", encoding="utf-8") as f:
    json.dump(out_data_w_entities_and_rel, f)

In [33]:
!(source ~/envs/perin-venv/bin/activate && cd ../../external_repos/direct_parsing_to_sent_graph/evaluation && python evaluate_single_dataset.py ../../../etl/data/processed/Perin_Preprocessing/01_test.json ../../../etl/data/processed/ORL_RE_Inference/01_test.json)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Source F1: 0.731
Target F1: 0.604
Expression F1: 0.800
Unlabeled Sentiment Tuple F1: 0.406
Sentiment Tuple F1: 0.278
{
  "source/f1": 0.7309854154541209,
  "target/f1": 0.6041468349975422,
  "expression/f1": 0.8002938109452079,
  "sentiment_tuple/unlabeled_f1": 0.4057627563970956,
  "sentiment_tuple/precision": 0.2773361480266674,
  "sentiment_tuple/recall": 0.27955722159168617,
  "sentiment_tuple/f1": 0.278442255623941
}

[0.7309854154541209, 0.6041468349975422, 0.8002938109452079, 0.4057627563970956, 0.2773361480266674, 0.27955722159168617, 0.278442255623941]
